In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
from functions import hyperSearch,createGraphRNN2,RNN_generator,init_state_update,RNN_forecast_Repeat,init_state_update_LSTM,RNN_forecast_Repeat_LSTM, LSTM2list
import cPickle
from hyperopt import hp, fmin, tpe

In [2]:
discreteList = ['dayOfWeek','payDay','month','earthquake','type','locale','locale_name','transferred','onpromotion']

In [3]:
learningRate = 1e-4
epoch = 10
bucketSize = 10000

In [4]:
cardinalitys_X = [55, 4001, 34, 337, 2, 23, 17, 6, 18]
cardinalitys_T = [7, 2, 13, 2, 7, 4, 25, 2, 3]
dimentions_X = [2, 20, 1, 2, 1, 1, 1, 1, 1]
dimentions_T = [1, 1, 1, 1, 1, 1, 1, 1, 1]

In [5]:
dX = sum(dimentions_X)
dT = sum(dimentions_T)
d = dX + dT + 2 # 2 for two cont variables

In [6]:
prefix = 'train'
y_np = np.loadtxt(prefix+'_Y',dtype=np.float32, delimiter=",") 
weight_np = np.loadtxt(prefix+'_Weight',dtype=np.float32, delimiter=",") 
Con_np = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Count_np = np.loadtxt(prefix+'_Count',dtype=np.int32,delimiter=",") 
Dis_np = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

In [7]:
prefix = 'val_SI'
y_np_val = np.loadtxt(prefix+'_Y',dtype=np.float32, delimiter=",") 
weight_np_val = np.loadtxt(prefix+'_Weight',dtype=np.float32, delimiter=",") 
Con_np_val = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np_val = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Count_np_val = np.loadtxt(prefix+'_Count',dtype=np.int32,delimiter=",") 
Dis_np_val = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

In [8]:
index = np.loadtxt('Index_val',dtype=np.int32,delimiter=",") 

In [9]:
space = hp.choice('batch_seq',[[100,16],[50,32],[25,64]]) + \
        [hp.uniform('keep_prob',0.4,0.9),hp.quniform('n_layers',2,5,1),\
         hp.uniform('grad_clip',1,10),\
         hp.choice('cell_type',['residual','highway','NormLSTM']),\
         hp.uniform('downsample',0.1,0.95),\
         hp.choice('optimizer',['SGD','Adam']),\
         hp.choice('actFun',['tanh','relu'])]

In [ ]:
best = fmin(hyperSearch,space,tpe.suggest,100)

INFO:tensorflow:Restoring parameters from RNN_fillin_01
loss:0.751857999259 ,batch_size:25 ,seq_len:64 ,keep_prob:0.438493811814 ,n_layers:4 ,grad_clip:1.44716967199 ,cell_type:residual ,downsample:0.190143634047 ,optimizer:Adam ,actFun:tanh 

INFO:tensorflow:Restoring parameters from RNN_fillin_01
loss:0.660828671164 ,batch_size:25 ,seq_len:64 ,keep_prob:0.428501799036 ,n_layers:3 ,grad_clip:4.2006247805 ,cell_type:NormLSTM ,downsample:0.734974933319 ,optimizer:Adam ,actFun:tanh 

INFO:tensorflow:Restoring parameters from RNN_fillin_01
loss:0.850416339561 ,batch_size:100 ,seq_len:16 ,keep_prob:0.684600786523 ,n_layers:3 ,grad_clip:2.46539967114 ,cell_type:NormLSTM ,downsample:0.370682558573 ,optimizer:SGD ,actFun:relu 

INFO:tensorflow:Restoring parameters from RNN_fillin_01
loss:0.837561625021 ,batch_size:100 ,seq_len:16 ,keep_prob:0.887715660492 ,n_layers:3 ,grad_clip:3.2011664133 ,cell_type:NormLSTM ,downsample:0.761040522817 ,optimizer:SGD ,actFun:relu 

INFO:tensorflow:Restoring 